In [19]:
import cv2
import numpy as np

# Constants
FRAME_WIDTH = 640
FRAME_HEIGHT = 480
MAX_NUM_OBJECTS = 5
MIN_OBJECT_AREA = 1
MAX_OBJECT_AREA = FRAME_HEIGHT * FRAME_WIDTH / 1.5

# Trackbar names and initial values
trackbar_values = {
    'H_MIN': 20,
    'H_MAX': 110,
    'S_MIN': 115,
    'S_MAX': 255,
    'V_MIN': 65,
    'V_MAX': 255
}
trackbar_window_name = "Trackbars"

def nothing(x):
    pass

def create_trackbars():
    cv2.namedWindow(trackbar_window_name)
    for trackbar, initial_value in trackbar_values.items():
        cv2.createTrackbar(trackbar, trackbar_window_name, initial_value, 256, nothing)

def draw_objects(contours, frame):
    for contour in contours:
        # Calculate moments for each contour
        # TODO: Could consider using this function
        # (x,y),r = cv2.minEnclosingCircle(contour)
        # print(f"Center:  {x},{y}")
        # print(r)
        moment = cv2.moments(contour)
        area = moment['m00']

        if MIN_OBJECT_AREA < area < MAX_OBJECT_AREA:
            # Calculate the center of the object
            x = int(moment['m10'] / area)
            y = int(moment['m01'] / area)

            # Fit an ellipse to the contour if possible
            if len(contour) >= 5:  # Need at least 5 points to fit an ellipse
                ellipse = cv2.fitEllipse(contour)
                center, axes, angle = ellipse
                radius = min(axes) / 2  # Use the minor axis length as the radius

                # Draw a circle around the detected object
                cv2.circle(frame, (int(center[0]), int(center[1])), int(radius), (0, 255, 0), 2)

                # Draw crosshairs
                cv2.line(frame, (int(center[0]), int(center[1])), (int(center[0]), max(int(center[1]) - 25, 0)), (0, 255, 0), 2)
                cv2.line(frame, (int(center[0]), int(center[1])), (int(center[0]), min(int(center[1]) + 25, FRAME_HEIGHT)), (0, 255, 0), 2)
                cv2.line(frame, (int(center[0]), int(center[1])), (max(int(center[0]) - 25, 0), int(center[1])), (0, 255, 0), 2)
                cv2.line(frame, (int(center[0]), int(center[1])), (min(int(center[0]) + 25, FRAME_WIDTH), int(center[1])), (0, 255, 0), 2)
                
                # Display coordinates and Radius
                cv2.putText(frame, f"{int(center[0])},{int(center[1])},{int(radius)}", (int(center[0]), int(center[1]) + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

def morph_ops(thresh):
    erode_element = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilate_element = cv2.getStructuringElement(cv2.MORPH_RECT, (8, 8))
    thresh = cv2.erode(thresh, erode_element, iterations=2)
    thresh = cv2.dilate(thresh, dilate_element, iterations=2)
    return thresh


# TODO: Get the radius and center of the ball. Will be used for distance estimation
# TODO: Use get_ball_location() to estimate the location of each detected ball
def track_filtered_objects(threshold, frame):
    temp = threshold.copy()
    contours, _ = cv2.findContours(temp, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        num_objects = len(contours)
        if num_objects < MAX_NUM_OBJECTS:
            draw_objects(contours, frame)
        else:
            cv2.putText(frame, "TOO MUCH NOISE! ADJUST FILTER", (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    


def get_ball_location(camera_matrix, detected_obj, robot_pose):
    # camera_matrix: (3x3 numpy array) with element 0,0 being the focal length x
    # detected_obj: (dict) Contains keys 'radius' and 'center', referring to the measured radius and pixel coordinates of the center of the ball
    # robot_pose:  (list) Contains the x,y,theta of the robot
    
    # TODO: Measure the actual radius of a tennis ball. This is what I found on google
    true_radius = 0.067 # Actual radius of tennis ball in meters 
    # TODO: Replace with width of image
    image_width = 0 # Width of image in number of pixels

    # Distance of ball based om measured radius 
    # distance = (r_true/r_measured) * foczl_length_x
    distance = true_radius/detected_obj['radius'] *camera_matrix[0][0] 

    # Estimate angle of object relative to robot
    theta = np.arctan((image_width/2 - detected_obj['center'])/ distance) 
    # theta = arctan(x_distance from the image center / distance based on measured radius)
    # Theta is +ve for objects to the left of the image center and -ve for objects to the right 
    
    # relative object location
    distance_obj = distance/np.cos(theta) # relative distance between robot and object
    x_relative = distance_obj * np.cos(theta) # relative x pose
    y_relative = distance_obj * np.sin(theta) # relative y pose

    # location of object in the world frame using rotation matrix
    delta_x_world = x_relative * np.cos(robot_pose[2]) - y_relative * np.sin(robot_pose[2])
    delta_y_world = x_relative * np.sin(robot_pose[2]) + y_relative * np.cos(robot_pose[2])

    # Get world coordinates of ball using robot pose
    ball_location = {'x':(robot_pose[0]+delta_x_world)[0], 
                     'y':(robot_pose[1]+delta_y_world)[0]}

    return ball_location

def main():
    create_trackbars()
    capture = cv2.VideoCapture(0)

    if not capture.isOpened():
        print("Error: Could not open video capture")
        return

    capture.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

    while True:
        ret, frame = capture.read()
        if not ret:
            print("Error: Empty frame captured")
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        hsv_values = {key: cv2.getTrackbarPos(key, trackbar_window_name) for key in trackbar_values}
        threshold = cv2.inRange(hsv, (hsv_values['H_MIN'], hsv_values['S_MIN'], hsv_values['V_MIN']),
                                         (hsv_values['H_MAX'], hsv_values['S_MAX'], hsv_values['V_MAX']))

        threshold = morph_ops(threshold)

        track_filtered_objects(threshold, frame)

        cv2.imshow("Thresholded Image", threshold)
        cv2.imshow("Original Image", frame)
        cv2.imshow("HSV Image", hsv)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


if __name__ == "__main__":
    main()
